In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from data_explore import viz

In [18]:
fname = '../data/SOCATv2024-1d_005deg-colloc-r20250224/SOCATv2024_1d_005deg_collocated_2010-r20250224.pq'
df = pd.read_parquet(fname)
df.head(5)

sst_deg_c        sal    pppp_hpa  \
expocode     time_1d    lat_005 lon_005                                     
06AQ20091129 2010-01-01 -55.525 203.875   6.191356  33.867355  987.126465   
                        -55.475 203.925   6.048000  33.838001  983.000000   
                                203.975   6.045000  33.839500  983.000000   
                        -55.425 203.975   6.039857  33.844143  983.000000   
                                204.025   6.059667  33.861111  983.000000   

                                           woa_sss  ncep_slp_hpa  \
expocode     time_1d    lat_005 lon_005                            
06AQ20091129 2010-01-01 -55.525 203.875  34.040001    988.900024   
                        -55.475 203.925  34.040001    988.900024   
                                203.975  34.040001    988.900024   
                        -55.425 203.975  34.040001    988.900024   
                                204.025  34.019001    988.900024   

                                         dist_to_land_km  fco2rec_uatm  \
expocode     time_1d    lat_005 lon_005                                  
06AQ20091129 2010-01-01 -55.525 203.875           1000.0    337.831977   
                        -55.475 203.925           1000.0    333.004000   
                                203.975           1000.0    333.633000   
                        -55.425 203.975           1000.0    335.474714   
                                204.025           1000.0    338.209889   

                                         fco2rec_flag  n_samples  \
expocode     time_1d    lat_005 lon_005                            
06AQ20091129 2010-01-01 -55.525 203.875             2         87   
                        -55.475 203.925             2          1   
                                203.975             2          4   
                        -55.425 203.975             2          7   
                                204.025             2          9   

                                         chl_globcolour  ...  time_sstcci  \
expocode     time_1d    lat_005 lon_005                  ...                
06AQ20091129 2010-01-01 -55.525 203.875        0.243971  ...   2010-01-01   
                        -55.475 203.925        0.235469  ...   2010-01-01   
                                203.975        0.251889  ...   2010-01-01   
                        -55.425 203.975        0.228635  ...   2010-01-01   
                                204.025        0.255812  ...   2010-01-01   

                                         lat_sstcci  lon_sstcci  sss_cci  \
expocode     time_1d    lat_005 lon_005                                    
06AQ20091129 2010-01-01 -55.525 203.875  -55.525002  203.875000      NaN   
                        -55.475 203.925  -55.474998  203.925003      NaN   
                                203.975  -55.474998  203.975006      NaN   
                        -55.425 203.975  -55.424999  203.975006      NaN   
                                204.025  -55.424999  204.024994      NaN   

                                         sss_cci_random_error  lat_ssscci  \
expocode     time_1d    lat_005 lon_005                                     
06AQ20091129 2010-01-01 -55.525 203.875                   NaN         NaN   
                        -55.475 203.925                   NaN         NaN   
                                203.975                   NaN         NaN   
                        -55.425 203.975                   NaN         NaN   
                                204.025                   NaN         NaN   

                                         lon_ssscci  time_ssscci  \
expocode     time_1d    lat_005 lon_005                            
06AQ20091129 2010-01-01 -55.525 203.875         NaN          NaT   
                        -55.475 203.925         NaN          NaT   
                                203.975         NaN          NaT   
                        -55.425 203.975         NaN          NaT   
       

# calculate distances between current ship position and last position

In [19]:
import numpy as np
from haversine import haversine_vector


expocodes = df.index.get_level_values('expocode').unique()
# cruise_id = expocodes[210]
cruise_id = expocodes[0]
cruise = df.loc[cruise_id].sort_values(by='time_avg')

In [20]:
print(expocodes.shape)

(315,)


In [13]:
72*20

1440

In [4]:
cruise.head()

lats = cruise.index.get_level_values('lat_005').to_numpy().reshape(-1, 1)
lons = cruise.index.get_level_values('lon_005').to_numpy().reshape(-1, 1)
coords = np.concatenate([lons, lats], axis=1)
coords_prev = np.zeros_like(coords)
coords_prev[1:] = coords[:-1]
distances = haversine_vector(coords, coords_prev, normalize=True)
distances[0] = -1

In [5]:
ddf = pd.DataFrame(distances, index=cruise.index, columns=['d'])
bins = [-1, 1,4,6,10, 20, 50, 100, 200, 2000, 5000, 10000]
binned = pd.cut(ddf.d, bins)
bin_counts = binned.value_counts().sort_index()
bin_counts

d
(-1, 1]           91
(1, 4]             1
(4, 6]           139
(6, 10]            0
(10, 20]          10
(20, 50]           8
(50, 100]          6
(100, 200]         0
(200, 2000]        0
(2000, 5000]       0
(5000, 10000]      0
Name: count, dtype: int64

In [14]:
ddft = pd.concat([ddf, cruise['time_avg'], pd.DataFrame(range(len(ddf)), index=ddf.index, columns=['ix'])], axis=1)
cruise_feat = ddft # select the cruise that we want with satellite temperature data
print(ddft.head())


map = viz.plot_cruise_interactive_scatter_map(cruise_feat.d, cmap='coolwarm', bins=20)
map  # uncomment to show the map

NameError: name 'ddf' is not defined

In [8]:
cruise_feat.iloc[[178, 179]]

d                   time_avg   ix
time_1d    lat_005 lon_005                                           
2015-04-16 25.325  278.375  67.053085 2015-04-16 07:00:39.071428  178
           24.975  278.925  61.436476 2015-04-16 07:07:52.000000  179

# Create training set
1. sort the entries for each cruise by time_avg to determine the coordinate sequence order, however there are some weird cases (2015, '33WA20150413')
2. calculate distances from every location to the location at the previous time step
3. for every entry, sum the distances up to that entry to derive total traveled distance
4. iterate through the locations and cut at 64*5 kms (or if two much time/space passed between two subsequent entries)
4. bin traveled distances in buckets of 5km width for each segment

In [9]:
from itertools import accumulate, chain
import numpy as np
from haversine import haversine_vector
import pandas as pd

def get_dtoprev(srt_cruise):
    # calculate distance in kms to previous locations
    coords = np.column_stack([
        srt_cruise.index.get_level_values('lon_005').to_numpy(),
        srt_cruise.index.get_level_values('lat_005').to_numpy()
    ])
    
    d = np.full(coords.shape[0], np.nan, dtype=np.float32)
    if coords.shape[0] > 1:
        d[1:] = haversine_vector(coords[1:], coords[:-1], normalize=True)
    
    return d

def divide_cruise(cruise, num_windows=64, len_window=5, max_time_delta=pd.Timedelta(days=1000), max_d_delta=np.inf):
    
    srt_cruise = cruise.sort_values(by='time_avg')
    d_diff = get_dtoprev(srt_cruise)
    time_diff = srt_cruise['time_avg'].diff()
    track_len= num_windows * len_window

    cs = 0
    segs = []
    cur_seg = [0]
    for (i, dprev) in enumerate(d_diff[1:]):
        cs += dprev
        # if the segment length exceeds 64*5 kms or the time jumps than max_time_delta or jumps more than max_d_delta kilometers
        # end the segment
        if cs >= track_len or time_diff.iloc[i] >= max_time_delta or dprev >= max_d_delta:
            segs.append(cur_seg)
            cur_seg = [0]
            cs = 0
        else:
            cur_seg.append(dprev.astype(np.float32))
    if cur_seg:
        segs.append(cur_seg)

    ix_segs = chain(*[[i]*len(seg) for (i, seg) in enumerate(segs)])
    cum_segs = chain(*[list(accumulate(seg)) for seg in segs])
    
    sdf = pd.DataFrame({'segment_id': list(ix_segs), 'track_length':list(cum_segs)}, index=srt_cruise.index)
    bins = np.arange(-len_window / 2., track_len + len_window, len_window)
    cut_sdf = pd.cut(sdf.track_length, bins=bins, labels=False)
    
    cruise['bin_segment_id'] = cut_sdf.values
    cruise['segment_id'] = sdf.segment_id
    return cruise

In [10]:
expocodes = df.index.get_level_values('expocode').unique()
# cruise_id = expocodes[1]
cruise_id = '06AQ20141203'
cruise = df.loc[cruise_id]
divide_cruise(cruise, num_windows=64, len_window=5, max_time_delta=pd.Timedelta(days=1))
map = viz.plot_cruise_interactive_scatter_map(cruise.segment_id ** ((cruise.segment_id % 2 == 0) * 100), cmap='viridis') # some dumb trick to get had color changes
# map = viz.plot_cruise_interactive_scatter_map(cruise['5kmtracks'], cmap='viridis')
map

C:\Users\henry\AppData\Local\Temp\ipykernel_8180\1113210358.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruise['bin_segment_id'] = cut_sdf.values
C:\Users\henry\AppData\Local\Temp\ipykernel_8180\1113210358.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruise['segment_id'] = sdf.segment_id


In [11]:
df2 = df.groupby(level=0, group_keys=False).apply(
    lambda cruise: divide_cruise(cruise,
                                 num_windows=64,
                                 len_window=5, 
                                 max_time_delta=pd.Timedelta(days=1)
                                 )
    )

In [12]:
df2.columns

Index(['sst_deg_c', 'sal', 'pppp_hpa', 'woa_sss', 'ncep_slp_hpa',
       'dist_to_land_km', 'fco2rec_uatm', 'fco2rec_flag', 'n_samples',
       'chl_globcolour', 'chl_globcolour_uncert', 'chl_globcolour_flags',
       'time_chlgc', 'lat_chlgc', 'lon_chlgc', 'temp_soda', 'salt_soda',
       'lon_soda', 'lat_soda', 'depth_soda', 'time_soda', 'ssh_adt', 'ssh_sla',
       'lat_sshcm', 'lon_sshcm', 'time_sshcm', 'sst_cci',
       'sst_cci_uncertainty', 'ice_cci', 'time_sstcci', 'lat_sstcci',
       'lon_sstcci', 'sss_cci', 'sss_cci_random_error', 'lat_ssscci',
       'lon_ssscci', 'time_ssscci', 'mld_dens_soda', 'time_avg',
       'bin_segment_id', 'segment_id'],
      dtype='object')

In [13]:
df2.head()
assert((df2.index == df.index).all())
print(df.index[:10])

MultiIndex([('06AQ20141203', '2015-01-01', -70.52499999999999, 351.925),
            ('06AQ20141203', '2015-01-02', -70.57499999999999, 350.925),
            ('06AQ20141203', '2015-01-02', -70.57499999999999, 350.975),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 350.975),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 351.025),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 351.075),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 351.925),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 351.975),
            ('06AQ20141203', '2015-01-02', -70.52499999999999, 352.025),
            ('06AQ20141203', '2015-01-02',            -70.475, 351.075)],
           names=['expocode', 'time_1d', 'lat_005', 'lon_005'])


In [16]:

df_grouped = df2.groupby([pd.Grouper(level=0), 'segment_id', 'bin_segment_id']).mean()
df_grouped.index.names, df2.index.names

(FrozenList(['expocode', 'segment_id', 'bin_segment_id']),
 FrozenList(['expocode', 'time_1d', 'lat_005', 'lon_005']))

In [213]:
def df_to_numpy(df2):
    binned = df2.groupby([pd.Grouper(level=0), 'segment_id', 'bin_segment_id']).mean() # first bin all values in the 5km buckets
    # index level <binned> : [expocode, segment_id, bin_segment_id]
    bins_per_seg = binned.groupby([pd.Grouper(level=0), pd.Grouper(level=1)]).size() # number of non-empty buckets per segment
    num_segs_tot = bins_per_seg.size # total number of segments in dataset
    segs_per_expocode = bins_per_seg.groupby(level=0).size() # number of segments per expocode
    
    offsets = segs_per_expocode.values.cumsum() # offsets to index segments in expocode table
    offsets_expocode = offsets - offsets[0] # offsets at expocode level
    offsets_seg = np.repeat(offsets_expocode, segs_per_expocode.values)
    offsets_seg = offsets_seg + bins_per_seg.index.get_level_values(level=1).to_numpy() # offset for each segment
    offsets_seg_per_bin = np.repeat(offsets_seg, bins_per_seg) # offset for each bin (row index in dataset does not change)

    dataset = np.full((num_segs_tot, 65), np.nan, dtype=np.float32)
    bin_ids = binned.index.get_level_values(level=2).to_numpy()
    dataset[offsets_seg_per_bin, bin_ids] = binned.fco2rec_uatm.values

    return dataset


    

In [214]:
df_to_numpy(df2)

array([[387.90848,       nan, 382.7484 , ...,       nan,       nan,
              nan],
       [295.94403, 276.03098,       nan, ...,       nan,       nan,
              nan],
       [348.27432,       nan,       nan, ..., 322.56458, 336.46738,
        356.4901 ],
       ...,
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan],
       [      nan,       nan,       nan, ...,       nan,       nan,
              nan]], shape=(14514, 65), dtype=float32)

In [145]:
df_grouped.head()

sst_deg_c        sal    pppp_hpa  \
expocode     segment_id bin_segment_id                                     
06AQ20141203 0          0               -1.628297  34.254623  992.383057   
                        2               -1.549885  34.321327  986.000000   
                        4               -0.854400  34.318066  986.000000   
                        6               -0.956750  34.309502  986.000000   
                        7               -1.110143  34.240715  983.714294   

                                          woa_sss  ncep_slp_hpa  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0               33.980877    991.667175   
                        2               33.818001    985.900024   
                        4               33.818001    985.900024   
                        6               33.818001    985.900024   
                        7               33.787998    983.799988   

                                        dist_to_land_km  fco2rec_uatm  \
expocode     segment_id bin_segment_id                                  
06AQ20141203 0          0                     67.000000    387.908463   
                        2                     66.770493    382.748426   
                        4                     68.266670    336.618267   
                        6                     69.500000    361.268250   
                        7                     83.571426    321.854857   

                                        fco2rec_flag  n_samples  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0                        2.0     1005.0   
                        2                        2.0       61.0   
                        4                        2.0       15.0   
                        6                        2.0        4.0   
                        7                        2.0        7.0   

                                        chl_globcolour  ...  time_sstcci  \
expocode     segment_id bin_segment_id                  ...                
06AQ20141203 0          0                     0.210151  ...   2015-01-01   
                        2                     0.203763  ...   2015-01-03   
                        4                     0.197458  ...   2015-01-03   
                        6                     0.214619  ...   2015-01-03   
                        7                     0.228982  ...   2015-01-03   

                                        lat_sstcci  lon_sstcci  sss_cci  \
expocode     segment_id bin_segment_id                                    
06AQ20141203 0          0               -70.525002  351.924988      NaN   
                        2               -70.574997  350.924988      NaN   
                        4               -70.574997  350.975006      NaN   
                        6               -70.525002  350.975006      NaN   
                        7               -70.425003  351.274994      NaN   

                                        sss_cci_random_error  lat_ssscci  \
expocode     segment_id bin_segment_id                                     
06AQ20141203 0          0                                NaN     -70.625   
                        2                                NaN     -70.625   
                        4                                NaN     -70.625   
                        6                                NaN     -70.625   
                        7                                NaN     -70.375   

                                        lon_ssscci  time_ssscci  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0                  351.875   2015-01-01   
                        2                  350.875   2015-01-02   
                        4                  350.875   2015-01-02   
                        6                  350.875   2015-01-02   
                        7                  351.375   2015-01-

In [195]:
df_segs = df_grouped.groupby([pd.Grouper(level=0), pd.Grouper(level=1)]).size()
num_segs_tot = df_segs.size
print(num_segs_tot)
df_segs

14514


expocode      segment_id
06AQ20141203  0             30
              1             22
              2             43
              3             40
              4             32
                            ..
PAT520151219  19            46
              20            10
PAT520151229  0             50
              1             48
              2             35
Length: 14514, dtype: int64

In [1]:
num_segs_expocode = df_segs.groupby(level=0).size()
offsets = num_segs_expocode.values.cumsum()
offsets = offsets - offsets[0]

NameError: name 'df_segs' is not defined

In [197]:
dataset = np.full((num_segs_tot, 65), np.nan, dtype=np.float32)
rep_offsets = np.repeat(offsets, num_segs_expocode.to_numpy())
rep_offsets


array([    0,     0,     0, ..., 14484, 14484, 14484], shape=(14514,))

In [200]:
segs_ids = df_segs.index.get_level_values(level=1).to_numpy()
segs_indices = segs_ids + rep_offsets
bin_segs_indices = np.repeat(segs_indices, df_segs.values)
df_grouped['row_index'] = bin_segs_indices

In [212]:
bin_ids = df_grouped.index.get_level_values(level=2).to_numpy()
dataset[bin_segs_indices, bin_ids] = df_grouped.fco2rec_uatm.values

dataset[50]

array([422.8451 , 417.99057, 418.454  ,       nan, 417.5498 , 417.6143 ,
       417.22122, 417.3547 , 417.22366, 417.40704, 417.44568,       nan,
       416.91327, 417.09857, 416.927  , 416.83926, 417.02682,       nan,
       418.18524, 418.34744, 418.45074, 418.70264, 418.52774, 418.5302 ,
       418.64786, 418.4645 , 418.24072,       nan, 418.193  , 418.5995 ,
       418.4621 ,       nan,       nan,       nan, 418.69513, 419.54926,
       419.1335 , 420.182  , 420.1306 , 420.0617 , 420.14026,       nan,
       420.1706 , 420.0063 , 420.9452 , 421.83084, 421.854  ,       nan,
       423.2661 , 423.6641 , 423.42667, 423.916  , 423.74023, 424.1955 ,
       425.03653, 425.05344, 425.9457 ,       nan, 425.74432, 426.35526,
             nan, 427.461  , 427.65442,       nan,       nan],
      dtype=float32)

In [217]:
d1 = df_to_numpy(df2)

In [227]:
np.allclose(d1, dataset, equal_nan=True)

True

In [201]:
df_grouped.head()

sst_deg_c        sal    pppp_hpa  \
expocode     segment_id bin_segment_id                                     
06AQ20141203 0          0               -1.628297  34.254623  992.383057   
                        2               -1.549885  34.321327  986.000000   
                        4               -0.854400  34.318066  986.000000   
                        6               -0.956750  34.309502  986.000000   
                        7               -1.110143  34.240715  983.714294   

                                          woa_sss  ncep_slp_hpa  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0               33.980877    991.667175   
                        2               33.818001    985.900024   
                        4               33.818001    985.900024   
                        6               33.818001    985.900024   
                        7               33.787998    983.799988   

                                        dist_to_land_km  fco2rec_uatm  \
expocode     segment_id bin_segment_id                                  
06AQ20141203 0          0                     67.000000    387.908463   
                        2                     66.770493    382.748426   
                        4                     68.266670    336.618267   
                        6                     69.500000    361.268250   
                        7                     83.571426    321.854857   

                                        fco2rec_flag  n_samples  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0                        2.0     1005.0   
                        2                        2.0       61.0   
                        4                        2.0       15.0   
                        6                        2.0        4.0   
                        7                        2.0        7.0   

                                        chl_globcolour  ...  lat_sstcci  \
expocode     segment_id bin_segment_id                  ...               
06AQ20141203 0          0                     0.210151  ...  -70.525002   
                        2                     0.203763  ...  -70.574997   
                        4                     0.197458  ...  -70.574997   
                        6                     0.214619  ...  -70.525002   
                        7                     0.228982  ...  -70.425003   

                                        lon_sstcci sss_cci  \
expocode     segment_id bin_segment_id                       
06AQ20141203 0          0               351.924988     NaN   
                        2               350.924988     NaN   
                        4               350.975006     NaN   
                        6               350.975006     NaN   
                        7               351.274994     NaN   

                                        sss_cci_random_error  lat_ssscci  \
expocode     segment_id bin_segment_id                                     
06AQ20141203 0          0                                NaN     -70.625   
                        2                                NaN     -70.625   
                        4                                NaN     -70.625   
                        6                                NaN     -70.625   
                        7                                NaN     -70.375   

                                        lon_ssscci  time_ssscci  \
expocode     segment_id bin_segment_id                            
06AQ20141203 0          0                  351.875   2015-01-01   
                        2                  350.875   2015-01-02   
                        4                  350.875   2015-01-02   
                        6                  350.875   2015-01-02   
                        7                  351.375   2015-01-03   

                                        mld_dens_soda  \
expocode     segment_id bin_segmen

In [124]:
import numpy as np
from haversine import haversine_vector

def get_dtoprev(srt_cruise):
    coords = np.column_stack([
        srt_cruise.index.get_level_values('lon_005').to_numpy(),
        srt_cruise.index.get_level_values('lat_005').to_numpy()
    ])
    
    d = np.full(coords.shape[0], np.nan, dtype=np.float32)
    if coords.shape[0] > 1:
        d[1:] = haversine_vector(coords[1:], coords[:-1], normalize=True)
    
    return d

srt_cruise = cruise.sort_values(by='time_avg')
d = get_dtoprev(srt_cruise)
print(np.nanmax(d))


67.053085


In [126]:

time_diff = srt_cruise['time_avg'].diff()
cs = 0
segs = []
cur_seg = [0]
threshold  = 64 * 5
for (i, dprev) in enumerate(d[1:]):
    cs += dprev
    if cs >= threshold or time_diff.iloc[i + 1] > pd.Timedelta(days=1):
        segs.append(cur_seg)
        cur_seg = [0]
        cs = 0
    else:
        cur_seg.append(dprev)
if cur_seg:
    segs.append(cur_seg)


In [130]:
i = 2
print(segs[i])
print(sum(segs[i]))

[0, np.float32(5.559754), np.float32(22.239016), np.float32(27.79877), np.float32(5.559754), np.float32(22.239016), np.float32(22.255), np.float32(5.559754), np.float32(5.559754), np.float32(5.6258974), np.float32(61.29826), np.float32(61.360184), np.float32(5.8169913), np.float32(28.114922), np.float32(33.623917)]
312.61096


In [145]:
from itertools import accumulate, chain
ix_segs = chain(*[[i]*len(seg) for (i, seg) in enumerate(segs)])
cum_segs = chain(*[list(accumulate(seg)) for seg in segs])

sdf = pd.DataFrame({'segment_id': list(ix_segs), 'track_length':list(cum_segs)}, index=cruise.index)
max_track = 64 * 5
bins = np.arange(-2.5, max_track + 5, 5)
print(bins)

cut_sdf = pd.cut(sdf.track_length, bins=bins, labels=False)

sdf.track_length.max()


[ -2.5   2.5   7.5  12.5  17.5  22.5  27.5  32.5  37.5  42.5  47.5  52.5
  57.5  62.5  67.5  72.5  77.5  82.5  87.5  92.5  97.5 102.5 107.5 112.5
 117.5 122.5 127.5 132.5 137.5 142.5 147.5 152.5 157.5 162.5 167.5 172.5
 177.5 182.5 187.5 192.5 197.5 202.5 207.5 212.5 217.5 222.5 227.5 232.5
 237.5 242.5 247.5 252.5 257.5 262.5 267.5 272.5 277.5 282.5 287.5 292.5
 297.5 302.5 307.5 312.5 317.5 322.5]


319.2693786621094

In [149]:
cut_sdf.head(100)

time_1d     lat_005  lon_005
2015-04-14  25.625   279.875     0
                     279.925     1
            25.575   279.875     2
            25.525   279.875     2
            25.475   279.875     3
                                ..
2015-04-15  25.375   277.825    10
                     277.775    11
            25.425   277.775    11
                     277.825    12
            25.475   277.825    12
Name: track_length, Length: 100, dtype: int64

In [151]:
print(len(bins))
map = viz.plot_cruise_interactive_scatter_map(sdf.segment_id, cmap='coolwarm', bins=len(bins))
map

66


# create horizonal tessellation
- every cell has 64*5 km width
- there is a row of cells every 0.5 degrees latitude
- latitude range: [-89.975, 89.975] (should be the same as some of the satellite data grid)

In [2]:
import numpy as np
import math

# mean earth radius - https://en.wikipedia.org/wiki/Earth_radius#Mean_radius
earth_radius = 6371.0088

lat_grid = np.arange(-89.975, 90, 0.05)
# for every latitude scale the circle on the earths surface parallel to the equator.
scaled_rads = np.cos(lat_grid / 360.0 * (2*math.pi)) * earth_radius
# for every fixed latitude calculate the length in longitudes of 5 kms
kms_in_lons = (5. / scaled_rads) / (2*math.pi) * 360
# cells per latitude
num_cells = 360 / kms_in_lons

In [6]:
grid = []
num_cells_ceil = num_cells.astype(int) + 1
# calculate longitudinal coordinates along each latitude
for (i, lon) in enumerate(kms_in_lons):
    # num_cells_ceil contains the number of 5km segments around the earth at a given latitude (with circles paralles to the equator)
    n_5k_segments = num_cells_ceil[i]
    coords_lat = np.zeros(n_5k_segments)
    coords_lat[:] = lon * np.arange(0, n_5k_segments, 1, dtype=np.float32)
    # resize array so we get the chunkc of 64 cells
    coords64 = np.nan * np.ones(64 * np.ceil((n_5k_segments / 64)).astype(int))
    coords64[:n_5k_segments] = coords_lat
    grid.append(coords64.reshape(-1, 64))

In [ ]:
# create lat lon tuples to used as pandas multindex
lat_index = []
block_index = []
for lat, lons in zip(lat_grid, grid):
    for i in range(len(lons)):
        lat_index += [(lat, lon) for lon in lons[i] if not np.isnan(lon)]
        block_index += [i] * len(lat_index)

In [41]:
multi_index = pd.MultiIndex.from_tuples(lat_index, names=['lat_005', 'lon'])
grid_df = pd.DataFrame(block_index, index=multi_index, columns=["block_index"])

In [ ]:
grid_df.head(10)

block_index
lat_005 lon                    
-89.975 0.000000              0
        103.054526            0
        206.109052            0
        309.163577            0
-89.925 0.000000              0
        34.351517             0
        68.703035             0
        103.054552            0
        137.406069            0
        171.757587            0

In [ ]:
import sys
sys.getsizeof(grid_df)

944904652

# Testing divide cruise random 

In [1]:
import pandas as pd
# fname = '../data/SOCATv2024-1d_005deg-colloc-r20250224/SOCATv2024_1d_005deg_collocated_2015-r20250224.pq'
year = 2015
fname = f'../data/SOCATv2024_raw-collocated-1982_2021/SOCATv2024_raw_r20250307-1982_2021/SOCATv2024v_collocated-{year}.pq'
df = pd.read_parquet(fname)
cruise_id = '06AQ20141203'
df['time_1d'] = df['time'].dt.round('D')
df.set_index(['expocode', 'time_1d','lon', 'lat'], inplace=True)
df.head(5)

sal  sst_deg_c  pppp_hpa  \
expocode     time_1d    lon      lat                                    
06AQ20141203 2015-01-01 351.9279 -70.533  34.253     -1.680     993.0   
                                 -70.533  34.248     -1.685     993.0   
                                 -70.533  34.244     -1.683     993.0   
                                 -70.533  34.241     -1.677     993.0   
                                 -70.533  34.241     -1.678     993.0   

                                          woa_sss  ncep_slp_hpa  \
expocode     time_1d    lon      lat                              
06AQ20141203 2015-01-01 351.9279 -70.533   33.788         992.2   
                                 -70.533   33.788         992.2   
                                 -70.533   33.788         992.2   
                                 -70.533   33.788         992.2   
                                 -70.533   33.788         992.2   

                                          dist_to_land_km  fco2rec_uatm  \
expocode     time_1d    lon      lat                                      
06AQ20141203 2015-01-01 351.9279 -70.533             67.0       394.458   
                                 -70.533             67.0       393.478   
                                 -70.533             67.0       394.124   
                                 -70.533             67.0       393.198   
                                 -70.533             67.0       393.068   

                                          fco2rec_flag                time  \
expocode     time_1d    lon      lat                                         
06AQ20141203 2015-01-01 351.9279 -70.533             2 2015-01-01 00:00:01   
                                 -70.533             2 2015-01-01 00:01:07   
                                 -70.533             2 2015-01-01 00:02:16   
                                 -70.533             2 2015-01-01 00:03:22   
                                 -70.533             2 2015-01-01 00:04:28   

                                          year  ...  sst_cci_uncertainty  \
expocode     time_1d    lon      lat            ...                        
06AQ20141203 2015-01-01 351.9279 -70.533  2015  ...                 0.37   
                                 -70.533  2015  ...                 0.37   
                                 -70.533  2015  ...                 0.37   
                                 -70.533  2015  ...                 0.37   
                                 -70.533  2015  ...                 0.37   

                                          ice_cci  time_sstcci lat_sstcci  \
expocode     time_1d    lon      lat                                        
06AQ20141203 2015-01-01 351.9279 -70.533      0.4   2015-01-01 -70.525002   
                                 -70.533      0.4   2015-01-01 -70.525002   
                                 -70.533      0.4   2015-01-01 -70.525002   
                                 -70.533      0.4   2015-01-01 -70.525002   
                                 -70.533      0.4   2015-01-01 -70.525002   

                                          lon_sstcci  sss_cci  \
expocode     time_1d    lon      lat                            
06AQ20141203 2015-01-01 351.9279 -70.533  351.924988      NaN   
                                 -70.533  351.924988      NaN   
                                 -70.533  351.924988      NaN   
                                 -70.533  351.924988      NaN   
                                 -70.533  351.924988      NaN   

                                          sss_cci_random_error  lat_ssscci  \
expocode     time_1d    lon      lat                                         
06AQ20141203 2015-01-01 351.9279 -70.533                   NaN     -70.625   
                                 -70.533                   NaN     -70.625   
                                 -70.533                   NaN     -70.625   
                                 -70.533                   NaN     -70.

In [4]:
from fco2dataset.ucruise import divide_cruise_random, divide_cruise
cruise = df.loc[cruise_id]
print(cruise.shape)
srt_cruise_random = divide_cruise_random(cruise, num_windows=64, len_window=5, max_time_delta=pd.Timedelta(days=1), rep_ratio=1)
srt_cruise = divide_cruise(cruise, num_windows=64, len_window=5, max_time_delta=pd.Timedelta(days=1))
print("number of segements in random cruise: ", srt_cruise_random['segment_id'].nunique())
print("number of segements in cruise: ", srt_cruise['segment_id'].nunique())

(29751, 39)
random_starts shape:  [  930   958  2243  6870  7734  9942 10905 11093 13695 16058 18155 19235
 20808 21670 24701 25117 25282 26542 27552]
number of segements in random cruise:  19
number of segements in cruise:  19


In [5]:
srt_cruise.head(5)

sal  sst_deg_c  pppp_hpa  woa_sss  \
time_1d    lon      lat                                             
2015-01-01 351.9279 -70.533  34.253     -1.680     993.0   33.788   
                    -70.533  34.248     -1.685     993.0   33.788   
                    -70.533  34.244     -1.683     993.0   33.788   
                    -70.533  34.241     -1.677     993.0   33.788   
                    -70.533  34.241     -1.678     993.0   33.788   

                             ncep_slp_hpa  dist_to_land_km  fco2rec_uatm  \
time_1d    lon      lat                                                    
2015-01-01 351.9279 -70.533         992.2             67.0       394.458   
                    -70.533         992.2             67.0       393.478   
                    -70.533         992.2             67.0       394.124   
                    -70.533         992.2             67.0       393.198   
                    -70.533         992.2             67.0       393.068   

                             fco2rec_flag                time  year  ...  \
time_1d    lon      lat                                              ...   
2015-01-01 351.9279 -70.533             2 2015-01-01 00:00:01  2015  ...   
                    -70.533             2 2015-01-01 00:01:07  2015  ...   
                    -70.533             2 2015-01-01 00:02:16  2015  ...   
                    -70.533             2 2015-01-01 00:03:22  2015  ...   
                    -70.533             2 2015-01-01 00:04:28  2015  ...   

                             time_sstcci  lat_sstcci  lon_sstcci sss_cci  \
time_1d    lon      lat                                                    
2015-01-01 351.9279 -70.533   2015-01-01  -70.525002  351.924988     NaN   
                    -70.533   2015-01-01  -70.525002  351.924988     NaN   
                    -70.533   2015-01-01  -70.525002  351.924988     NaN   
                    -70.533   2015-01-01  -70.525002  351.924988     NaN   
                    -70.533   2015-01-01  -70.525002  351.924988     NaN   

                             sss_cci_random_error  lat_ssscci  lon_ssscci  \
time_1d    lon      lat                                                     
2015-01-01 351.9279 -70.533                   NaN     -70.625     351.875   
                    -70.533                   NaN     -70.625     351.875   
                    -70.533                   NaN     -70.625     351.875   
                    -70.533                   NaN     -70.625     351.875   
                    -70.533                   NaN     -70.625     351.875   

                             time_ssscci  bin_id  segment_id  
time_1d    lon      lat                                       
2015-01-01 351.9279 -70.533   2015-01-01       0           0  
                    -70.533   2015-01-01       0           0  
                    -70.533   2015-01-01       0           0  
                    -70.533   2015-01-01       0           0  
                    -70.533   2015-01-01       0           0  

[5 rows x 41 columns]

In [11]:
from data_explore import viz
vis_cruise = srt_cruise_random[srt_cruise_random['segment_id'] ==9]
map = viz.plot_cruise_interactive_scatter_map(vis_cruise.bin_id, cmap='coolwarm', lat_name='lat', lon_name='lon', time_name='time_1d',bins=20) # some dumb trick to get had color changes
map

In [ ]:
map = viz.plot_cruise_interactive_scatter_map(srt_cruise.segment_id, cmap='coolwarm', lat_name='lat', lon_name='lon', time_name='time_1d',bins=20) # some dumb trick to get had color changes
map

In [ ]:
map = viz.plot_cruise_interactive_scatter_map(cruise.fco2rec_uatm, cmap='coolwarm', lat_name='lat', lon_name='lon', time_name='time_1d',bins=20) # some dumb trick to get had color changes
map